In [ ]:
! pip install langchain umap-learn scikit-learn langchain_community tiktoken langchain-openai langchainhub chromadb langchain-anthropic

In [ ]:
! pip install langchain_experimental

In [ ]:
#from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

#query = "Hayao Miyazaki"
#raw_documents = WikipediaLoader(query=query).load()

loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
# create doc_text as list to store text
doc_text = [d.page_content for d in raw_documents]

In [ ]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer

# load Diffbot Token for later entity and relationship extraction
diffbot_api_key = "Diffbot-token"
os.environ["DIFFBOT_API_KEY"] = diffbot_api_key
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

# use Diffbot Graph Transformer to extract entities/relationships from Wikipedia articles
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [ ]:
# connect to our neo4j database
from langchain_community.graphs import Neo4jGraph

url = "neo4j-url"
username = "neo4j"
password = "database-pw"

graph = Neo4jGraph(url=url, username=username, password=password)

In [ ]:
# import extracted entities and relationships into neo4j graph database
graph.add_graph_documents(graph_documents)

In [ ]:
import matplotlib.pyplot as plt
import tiktoken
cnt = [num_tokens_from_string(d, "cl100k_base") for d in doc_text]

# Plotting the histogram of token counts
plt.figure(figsize=(10, 6))
plt.hist(cnt, bins=30, color="blue", edgecolor="black", alpha=0.7)
plt.title("Histogram of Token Counts")
plt.xlabel("Token Count")
plt.ylabel("Frequency")
plt.grid(axis="y", alpha=0.75)

# Display the histogram
plt.show

In [ ]:
# Doc texts concatenated all articles
d_sorted = sorted(raw_documents, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content1 = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Num tokens in all context: %s"
    % num_tokens_from_string(concatenated_content1, "cl100k_base")
)

In [ ]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk_size_tok = 2000
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=chunk_size_tok, chunk_overlap=0
# )
# texts_split = text_splitter.split_text(concatenated_content1)

In [ ]:
from langchain_openai import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = 'openai-api-key'
os.environ["ANTHROPIC_API_KEY"] = 'anthropic-api-key'
embd = OpenAIEmbeddings()

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(temperature=0, model="claude-3-opus-20240229")